In [1]:
import pandas,json,numpy

import scipy,scipy.stats

import matplotlib,matplotlib.pyplot

In [2]:
def histogrammer(x):

    # get number of bins based on Rice's rule
    rice=int((len(x)**(1/3))*2)
    print('\t number of bins according to Rice: {}'.format(rice))

    counts,edges=numpy.histogram(x,bins=rice)
    half=(edges[1]-edges[0])/2
    centers=edges[:-1]+half

    return centers,counts

# 0. user defined variables

In [3]:
expression_file='/Volumes/GoogleDrive/My Drive/projects/MINER/data/MMRF_CoMMpass_IA13a_E74GTF/MMRF_CoMMpass_IA13a_E74GTF_Salmon_Gene_TPM.txt'
transcriptional_states_file='/Users/alomana/Google Drive File Stream/My Drive/projects/MINER/shared/MINER/results_minCorrelation_0o2_50_allFiles/transcriptional_states.json'

# 1. read data

## 1.1. read expression data

In [4]:
expression=pandas.read_csv(expression_file,header=0,index_col=0,sep="\t")
print(expression.shape)
expression.head()

(57997, 892)


,MMRF_1775_1_BM,MMRF_1407_1_BM,MMRF_1358_1_BM,MMRF_2813_1_BM,MMRF_2341_1_BM,MMRF_1380_2_BM,MMRF_2716_1_BM,MMRF_1710_1_BM,MMRF_1839_1_BM,MMRF_1755_1_BM,...,MMRF_2141_1_BM,MMRF_1941_1_BM,MMRF_1331_1_BM,MMRF_1327_1_BM,MMRF_1944_1_BM,MMRF_1915_1_BM,MMRF_2143_1_BM,MMRF_1683_1_BM,MMRF_1957_2_BM,MMRF_1773_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,1.241630,1.83582,0.425776,4.987650,0.261943,6.68659,0.807806,4.31860,0.747555,0.375819,...,0.531679,1.206830,1.555390,0.736861,0.023523,0.015962,3.732960,3.60618,0.039907,0.036809
ENSG00000000005,0.008621,0.03133,0.000000,0.000000,0.031981,0.00000,0.000000,0.00000,0.000000,0.011938,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062777,0.00000,0.000000,0.000000
ENSG00000000419,18.235300,16.67170,18.409500,17.192400,27.803200,24.99560,11.764300,20.03830,16.482000,42.492300,...,14.521200,12.468100,14.442600,25.120900,26.061200,36.432700,11.700700,15.07760,14.912000,50.370700
ENSG00000000457,1.368570,2.39387,0.857731,0.725198,4.810990,4.65214,1.406890,3.89088,3.530600,6.051540,...,2.911840,1.955650,2.589970,5.895050,2.274490,1.747270,1.257150,5.66744,1.058240,2.746610
ENSG00000000460,0.284560,2.20924,0.345042,0.254061,2.895470,1.52748,0.298312,2.96181,1.153110,2.243760,...,0.726033,0.435713,0.430427,3.994530,1.100370,1.355290,0.417126,2.05526,0.688754,1.633540


## 1.2. obtain model state labels

In [5]:
# read data
with open(transcriptional_states_file) as json_file:
    states = json.load(json_file)

In [6]:
# remove peripheral blood samples
bm_states={}
for state_ID in states:
    bm=[element for element in states[state_ID] if element.split('_')[-1] == 'BM']
    bm_states[state_ID]=bm

In [7]:
# get a list of BM patients
all_patients=[]
for state_ID in bm_states:
    for patient in bm_states[state_ID]:
        if patient not in all_patients:
            all_patients.append(patient)
print(len(all_patients))

821


In [8]:
# intersect with count headers
intersect=list(set(expression.columns) & set(all_patients))
print(len(intersect))

821


In [9]:
# define states
states={}
for state in bm_states:
    states[state]=[]
    for patient in bm_states[state]:
        if patient in intersect:
            states[state].append(patient)

In [10]:
# define sizes
sizes=[]
for state in bm_states:
    sizes.append(len(states[state]))
sizes.sort(reverse=True)

# 2. perform analysis

In [11]:
# work with states with at least 50 patients
selected_states={}
selected_patients=[]
for key in states:
    if len(states[key]) > 50:
        print(key,len(states[key]))
        selected_states[key]=[]
        for element in states[key]:
            selected_states[key].append(element)
            selected_patients.append(element)

1 96
0 104
3 92
11 54
16 62


## 2.0. manipulate expression

In [12]:
# get rid of not mapping patients
expression=expression[selected_patients]
print(expression.shape)
expression.head()

(57997, 408)


,MMRF_2720_1_BM,MMRF_1311_1_BM,MMRF_2400_1_BM,MMRF_1501_1_BM,MMRF_2473_1_BM,MMRF_2734_1_BM,MMRF_2816_1_BM,MMRF_2764_1_BM,MMRF_2770_1_BM,MMRF_1698_1_BM,...,MMRF_2524_1_BM,MMRF_2532_1_BM,MMRF_2057_1_BM,MMRF_1652_1_BM,MMRF_2379_1_BM,MMRF_1867_1_BM,MMRF_1888_1_BM,MMRF_1991_1_BM,MMRF_1689_1_BM,MMRF_2001_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,0.090582,0.067477,0.00000,0.181269,3.20756,0.381557,0.322808,6.721010,7.36350,0.028807,...,0.11732,1.075420,0.113491,3.951350,0.013601,3.679860,5.58479,0.034003,0.797677,0.045272
ENSG00000000005,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00000,0.012746,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
ENSG00000000419,43.135000,22.682000,29.97700,21.867000,11.72560,17.219000,12.853300,9.937110,22.74100,9.413130,...,16.45950,24.671200,24.333900,21.392900,23.577100,22.326200,21.83850,18.602400,34.232100,12.935100
ENSG00000000457,2.146630,3.947790,2.77676,1.529750,2.96216,1.872750,2.520000,1.910060,2.96440,0.617233,...,2.86611,3.758220,2.654060,2.932530,2.328460,2.052120,3.14741,3.309700,4.706580,3.144820
ENSG00000000460,4.627430,1.826550,1.69400,1.009420,1.38902,0.523769,2.142790,0.886782,1.63542,0.146540,...,1.35645,1.803110,1.012110,0.799479,1.573360,0.310039,1.24096,0.713808,2.258240,1.230020


In [13]:
# remove genes that do not reach 10 TPMs
expression=expression[(expression > 10).any(axis=1)]
print(expression.shape)
expression.head()

(13386, 408)


,MMRF_2720_1_BM,MMRF_1311_1_BM,MMRF_2400_1_BM,MMRF_1501_1_BM,MMRF_2473_1_BM,MMRF_2734_1_BM,MMRF_2816_1_BM,MMRF_2764_1_BM,MMRF_2770_1_BM,MMRF_1698_1_BM,...,MMRF_2524_1_BM,MMRF_2532_1_BM,MMRF_2057_1_BM,MMRF_1652_1_BM,MMRF_2379_1_BM,MMRF_1867_1_BM,MMRF_1888_1_BM,MMRF_1991_1_BM,MMRF_1689_1_BM,MMRF_2001_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,0.090582,0.067477,0.000000,0.181269,3.20756,0.381557,0.322808,6.721010,7.36350,0.028807,...,0.117320,1.075420,0.113491,3.951350,0.013601,3.679860,5.584790,0.034003,0.797677,0.045272
ENSG00000000419,43.135000,22.682000,29.977000,21.867000,11.72560,17.219000,12.853300,9.937110,22.74100,9.413130,...,16.459500,24.671200,24.333900,21.392900,23.577100,22.326200,21.838500,18.602400,34.232100,12.935100
ENSG00000000457,2.146630,3.947790,2.776760,1.529750,2.96216,1.872750,2.520000,1.910060,2.96440,0.617233,...,2.866110,3.758220,2.654060,2.932530,2.328460,2.052120,3.147410,3.309700,4.706580,3.144820
ENSG00000000460,4.627430,1.826550,1.694000,1.009420,1.38902,0.523769,2.142790,0.886782,1.63542,0.146540,...,1.356450,1.803110,1.012110,0.799479,1.573360,0.310039,1.240960,0.713808,2.258240,1.230020
ENSG00000000938,0.098370,0.945676,0.668806,0.030880,0.74193,0.222996,0.481341,0.737870,1.42656,0.107661,...,0.173866,0.134758,0.079931,0.084605,0.188103,0.163968,0.180659,0.021796,0.851161,0.052441


In [14]:
# transform to ranks
ranks=expression.rank(ascending=False)
print(ranks.shape)
ranks.head()

(13386, 408)


,MMRF_2720_1_BM,MMRF_1311_1_BM,MMRF_2400_1_BM,MMRF_1501_1_BM,MMRF_2473_1_BM,MMRF_2734_1_BM,MMRF_2816_1_BM,MMRF_2764_1_BM,MMRF_2770_1_BM,MMRF_1698_1_BM,...,MMRF_2524_1_BM,MMRF_2532_1_BM,MMRF_2057_1_BM,MMRF_1652_1_BM,MMRF_2379_1_BM,MMRF_1867_1_BM,MMRF_1888_1_BM,MMRF_1991_1_BM,MMRF_1689_1_BM,MMRF_2001_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,12219.0,12523.0,13191.0,11880.0,8054.0,11590.0,11817.0,6201.0,6312.0,12137.0,...,12017.0,10901.5,11779.0,7859.0,12671.0,7948.0,7574.0,12430.0,11467.0,12315.0
ENSG00000000419,1883.0,3176.0,3271.0,2371.0,3351.0,2829.0,4346.0,4857.0,2835.0,2236.0,...,3036.0,2894.0,1606.0,2286.0,1989.0,2307.0,2724.0,2848.0,2394.0,3961.5
ENSG00000000457,10404.0,9081.0,10233.0,10036.0,8332.0,9886.0,9621.0,9858.0,9151.0,9810.0,...,8952.0,9048.0,8245.0,8718.0,9074.0,9511.0,9191.0,8666.0,9079.0,8755.0
ENSG00000000460,9034.0,10513.0,10872.0,10623.0,10298.0,11370.0,9993.0,11003.0,10368.0,11296.0,...,10334.0,10316.0,10141.0,10791.0,9887.0,11459.0,10567.0,10895.0,10456.0,10306.0
ENSG00000000938,12190.0,11170.0,11625.0,12550.0,11150.0,11917.0,11599.0,11182.0,10567.0,11506.0,...,11833.0,12107.0,11944.0,12079.0,11687.0,11855.0,11615.0,12552.0,11423.0,12271.0


## 2.1. Spearman correlation

In [15]:
# sort labels in order. not important, just for aestetic purposes
state_labels=list(selected_states.keys())
state_labels=[int(label) for label in state_labels]
state_labels.sort()
state_labels=[str(label) for label in state_labels]
print(state_labels)

['0', '1', '3', '11', '16']


In [16]:
# find the average ranking of each state
for state in state_labels:
    average_label='average_state_'+state
    ranks[average_label]=ranks[selected_states[state]].median(axis=1)
print(ranks.shape)
ranks.head()

(13386, 413)


,MMRF_2720_1_BM,MMRF_1311_1_BM,MMRF_2400_1_BM,MMRF_1501_1_BM,MMRF_2473_1_BM,MMRF_2734_1_BM,MMRF_2816_1_BM,MMRF_2764_1_BM,MMRF_2770_1_BM,MMRF_1698_1_BM,...,MMRF_1867_1_BM,MMRF_1888_1_BM,MMRF_1991_1_BM,MMRF_1689_1_BM,MMRF_2001_1_BM,average_state_0,average_state_1,average_state_3,average_state_11,average_state_16
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,12219.0,12523.0,13191.0,11880.0,8054.0,11590.0,11817.0,6201.0,6312.0,12137.0,...,7948.0,7574.0,12430.0,11467.0,12315.0,11085.0,11999.5,11671.5,11446.0,11623.0
ENSG00000000419,1883.0,3176.0,3271.0,2371.0,3351.0,2829.0,4346.0,4857.0,2835.0,2236.0,...,2307.0,2724.0,2848.0,2394.0,3961.5,2211.0,2664.5,3023.5,2556.0,2812.5
ENSG00000000457,10404.0,9081.0,10233.0,10036.0,8332.0,9886.0,9621.0,9858.0,9151.0,9810.0,...,9511.0,9191.0,8666.0,9079.0,8755.0,9717.5,9231.0,9057.0,8677.5,8870.0
ENSG00000000460,9034.0,10513.0,10872.0,10623.0,10298.0,11370.0,9993.0,11003.0,10368.0,11296.0,...,11459.0,10567.0,10895.0,10456.0,10306.0,11132.5,10764.0,10988.0,10297.5,10776.0
ENSG00000000938,12190.0,11170.0,11625.0,12550.0,11150.0,11917.0,11599.0,11182.0,10567.0,11506.0,...,11855.0,11615.0,12552.0,11423.0,12271.0,10248.0,11421.5,11815.0,11721.0,11774.5


In [17]:
average_ranks=ranks.iloc[:,-len(state_labels):]
ranks.drop(columns=average_ranks.columns,inplace=True)

In [18]:
print(average_ranks.shape)
average_ranks.head()

(13386, 5)


,average_state_0,average_state_1,average_state_3,average_state_11,average_state_16
GENE_ID,,,,,
ENSG00000000003,11085.0,11999.5,11671.5,11446.0,11623.0
ENSG00000000419,2211.0,2664.5,3023.5,2556.0,2812.5
ENSG00000000457,9717.5,9231.0,9057.0,8677.5,8870.0
ENSG00000000460,11132.5,10764.0,10988.0,10297.5,10776.0
ENSG00000000938,10248.0,11421.5,11815.0,11721.0,11774.5


In [19]:
print(ranks.shape)
ranks.head()

(13386, 408)


,MMRF_2720_1_BM,MMRF_1311_1_BM,MMRF_2400_1_BM,MMRF_1501_1_BM,MMRF_2473_1_BM,MMRF_2734_1_BM,MMRF_2816_1_BM,MMRF_2764_1_BM,MMRF_2770_1_BM,MMRF_1698_1_BM,...,MMRF_2524_1_BM,MMRF_2532_1_BM,MMRF_2057_1_BM,MMRF_1652_1_BM,MMRF_2379_1_BM,MMRF_1867_1_BM,MMRF_1888_1_BM,MMRF_1991_1_BM,MMRF_1689_1_BM,MMRF_2001_1_BM
GENE_ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,12219.0,12523.0,13191.0,11880.0,8054.0,11590.0,11817.0,6201.0,6312.0,12137.0,...,12017.0,10901.5,11779.0,7859.0,12671.0,7948.0,7574.0,12430.0,11467.0,12315.0
ENSG00000000419,1883.0,3176.0,3271.0,2371.0,3351.0,2829.0,4346.0,4857.0,2835.0,2236.0,...,3036.0,2894.0,1606.0,2286.0,1989.0,2307.0,2724.0,2848.0,2394.0,3961.5
ENSG00000000457,10404.0,9081.0,10233.0,10036.0,8332.0,9886.0,9621.0,9858.0,9151.0,9810.0,...,8952.0,9048.0,8245.0,8718.0,9074.0,9511.0,9191.0,8666.0,9079.0,8755.0
ENSG00000000460,9034.0,10513.0,10872.0,10623.0,10298.0,11370.0,9993.0,11003.0,10368.0,11296.0,...,10334.0,10316.0,10141.0,10791.0,9887.0,11459.0,10567.0,10895.0,10456.0,10306.0
ENSG00000000938,12190.0,11170.0,11625.0,12550.0,11150.0,11917.0,11599.0,11182.0,10567.0,11506.0,...,11833.0,12107.0,11944.0,12079.0,11687.0,11855.0,11615.0,12552.0,11423.0,12271.0


In [20]:
average_ranks.corr()

,average_state_0,average_state_1,average_state_3,average_state_11,average_state_16
average_state_0,1.000000,0.974862,0.933742,0.948506,0.926725
average_state_1,0.974862,1.000000,0.979362,0.969876,0.975344
average_state_3,0.933742,0.979362,1.000000,0.961048,0.980307
average_state_11,0.948506,0.969876,0.961048,1.000000,0.955678
average_state_16,0.926725,0.975344,0.980307,0.955678,1.000000


In [28]:
# predict labels of each patient
# iterate over state to know reference
predicted_labels={}
true_labels={}

for patient in selected_patients:
    
    # compute Spearman-rank correlation to each average
    subset=average_ranks.join(ranks[patient],how='left')

    
    print(patient,subset.shape)
    print(subset.shape)
    print(subset.head())
    print(subset.corr())
    break

MMRF_2720_1_BM (13386, 6)
(13386, 6)
                 average_state_0  average_state_1  average_state_3  \
GENE_ID                                                              
ENSG00000000003          11085.0          11999.5          11671.5   
ENSG00000000419           2211.0           2664.5           3023.5   
ENSG00000000457           9717.5           9231.0           9057.0   
ENSG00000000460          11132.5          10764.0          10988.0   
ENSG00000000938          10248.0          11421.5          11815.0   

                 average_state_11  average_state_16  MMRF_2720_1_BM  
GENE_ID                                                              
ENSG00000000003           11446.0           11623.0         12219.0  
ENSG00000000419            2556.0            2812.5          1883.0  
ENSG00000000457            8677.5            8870.0         10404.0  
ENSG00000000460           10297.5           10776.0          9034.0  
ENSG00000000938           11721.0           11774.5 

In [ ]:
# find the average ranking per state, do a histogram per state
# find the intra/inter SRC 

## 2.2. gene pairs

## 2.3. state descriptors

In [ ]:
# values=df.iloc[:,:2].to_numpy()
# log2_values=numpy.log2(values+1)
# print(len(values))
# # centers,counts=histogrammer(log2_values)
# print(centers,counts)
# matplotlib.pyplot.plot(centers,counts,'o-',color='black')
# average=numpy.median(log2_values)
# deviation=numpy.std(log2_values)
# lb=average-deviation
# ub=average+deviation
# print(average,deviation)
# matplotlib.pyplot.axvline(average,color='red')
# matplotlib.pyplot.axvline(lb,color='orange')
# matplotlib.pyplot.axvline(ub,color='orange')
# matplotlib.pyplot.axvline(numpy.log10(1+1),color='blue')